# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import

import numpy as np
import pandas as pd
import scipy.stats

from scipy.stats import ttest_1samp
from statsmodels.stats.proportion import proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [3]:
data = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')
data.rename(columns=lambda c: c.lower().replace(' ', '_').replace('-', '_'), inplace=True)

In [4]:
data.nunique()

name                 33380
job_titles            1119
department              36
full_or_part_time        2
salary_or_hourly         2
typical_hours            4
annual_salary          924
hourly_rate            178
dtype: int64

Examine the `salaries` dataset using the `head` function below.

In [5]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
data.isna().sum()

name                     0
job_titles               0
department               0
full_or_part_time        0
salary_or_hourly         0
typical_hours        25528
annual_salary         8174
hourly_rate          25528
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
nb_salaried = len(data[data['salary_or_hourly'] == 'Salary'])
nb_hourly = len(data[data['salary_or_hourly'] == 'Hourly'])

print(f'There are {nb_salaried} salaried and {nb_hourly} hourly')

There are 25528 salaried and 8174 hourly


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
data.pivot_table(index='department', values='name', aggfunc=pd.Series.nunique).reset_index().sort_values('name', ascending=False)

,department,name
27,POLICE,13784
16,FIRE,4615
32,STREETS & SAN,2170
35,WATER MGMNT,1912
2,AVIATION,1855
26,OEMC,1737
33,TRANSPORTN,1111
30,PUBLIC LIBRARY,1088
12,DAIS,1032
14,FAMILY & SUPPORT,655


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
hourly_workers_rates = data[data['salary_or_hourly'] == 'Hourly']['hourly_rate']
hourly_workers_rates.head()

2     13.00
5     37.45
15    40.78
16    38.52
21    34.98
Name: hourly_rate, dtype: float64

In [ ]:
# H0: mean hourly rate = 30$
# H1: mean hourly rate <> 30$

In [10]:
stat, pval = ttest_1samp(hourly_workers_rates, 30)

print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)

stat is   31.211500329216044
pvalue for the two-tailed test is  3.825475847673794e-202


In [11]:
# P value is low so H0 (hourly rates are equal to 30) can be rejected. Stats gives us a standard error of the mean of 31
# Hourly rates can be considered different than 30$/h

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [ ]:
# H0: mean salary for police force >= 86000$
# H1: mean salary for police force < 86000$

In [12]:
police_dep_salaries = data[(data['department'] == 'POLICE') & (data['salary_or_hourly'] == 'Salary')]['annual_salary']
stat, pval = ttest_1samp(police_dep_salaries, 86000)

print('stat is  ', stat)
print('pvalue for the one-tailed test is ', pval/2)

stat is   18.51748561981861
pvalue for the one-tailed test is  6.119566952121482e-76


In [13]:
# P value is low so the hypothesis salaries are lower to 86000$/year can be considered false. Salaries can be considered higher or equal to 86000$/year.

In [14]:
confidence_level = 0.95
degrees_freedom = len(police_dep_salaries) - 1
sample_mean = np.mean(police_dep_salaries)
sample_standard_error = scipy.stats.sem(police_dep_salaries)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)
                                             
print( 'confidence interval is ', confidence_interval, '.' )

confidence interval is  (88534.1182885883, 89134.11955863392) .


In [ ]:
# We verified with the confidence interval that salaries are between 88534$ and 89134$.

Using the `crosstab` function, find the department that has the most hourly workers. 

In [15]:
hourly_workers = data[data['salary_or_hourly'] == 'Hourly']

data_crosstab = pd.crosstab(hourly_workers['department'], hourly_workers['hourly_rate'], margins=True, margins_name='nb_workers', values=hourly_workers['name'], aggfunc=pd.Series.nunique).reset_index()[['department', 'nb_workers']].sort_values("nb_workers", ascending=False)
data_crosstab

hourly_rate,department,nb_workers
22,nb_workers,8139
19,STREETS & SAN,1842
21,WATER MGMNT,1556
1,AVIATION,1214
14,OEMC,1077
6,DAIS,744
20,TRANSPORTN,724
8,FAMILY & SUPPORT,356
17,PUBLIC LIBRARY,356
4,CITY COUNCIL,74


In [16]:
# STREETS & SAN has the most hourly workers

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [ ]:
# H0: hourly rate >= 35$
# H1: hourly rate < 35$

In [17]:
street_san_hourly_workers_rates = data[(data['department'] == 'STREETS & SAN') & (data['salary_or_hourly'] == 'Hourly')]['hourly_rate']
stat, pval = ttest_1samp(street_san_hourly_workers_rates, 35)

print('stat is  ', stat)
print('pvalue for the one-tailed test is ', pval/2)

stat is   4.000138803968834
pvalue for the one-tailed test is  3.2902372218867895e-05


In [18]:
# P Value is low so STREETS & SAN workers earn indeed less than 35$/hour

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [19]:
confidence_level = 0.95

In [20]:
degrees_freedom = len(hourly_workers_rates) - 1
sample_mean = np.mean(hourly_workers_rates)
sample_standard_error = scipy.stats.sem(hourly_workers_rates)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)
                                             
print( 'confidence interval is ', confidence_interval, '.' )

confidence interval is  (34.29839539204361, 34.874504045197305) .


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [21]:
police_dep_salaries = data[(data['department'] == 'POLICE') & (data['salary_or_hourly'] == 'Salary')]['annual_salary']

degrees_freedom = len(police_dep_salaries) - 1
sample_mean = np.mean(police_dep_salaries)
sample_standard_error = scipy.stats.sem(police_dep_salaries)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)
                                             
print( 'confidence interval is ', confidence_interval, '.' )

confidence interval is  (88534.1182885883, 89134.11955863392) .


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [25]:
stat, pval = proportions_ztest(count=len(hourly_workers), nobs=len(data), value = 0.25)
print('stat: %0.3f, value: %0.3f' % (stat, pval))

stat: -3.196, value: 0.001


In [28]:
# We can reject the fact that number of hourly workers in Chicago is significantly different from 25% at the 95% confidence level. In fact, it is equal to it.